<a href="https://colab.research.google.com/github/AndreaFabro/Tech_fase3/blob/main/finetuning_split_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [30]:
# instalando o Hugging Face Transformers e dataset
!pip install transformers datasets

In [20]:
import json
from datasets import Dataset

# Caminho do dataset no Google Drive
INPUT_PATH_DATASET = '/content/drive/MyDrive/finetuning/split_titles_dataset_1.json'

# 1. Carregar o dataset original
with open(INPUT_PATH_DATASET, 'r', encoding='utf-8') as input_file:
    data = json.load(input_file)

# 2. Pré-processar o campo 'input' para garantir que é uma string
def preprocess_input(file):
    if isinstance(file['input'], list):  # Se for uma lista, unir os elementos em uma string
        file['input'] = " ".join(file['input'])
    return file

# Aplicar a função de pré-processamento a cada item do dataset
preprocessed_data = [preprocess_input(file) for file in data]

# 3. Converter o dataset para o formato Hugging Face
dataset = Dataset.from_list(preprocessed_data)

# Verificar o dataset processado
print(dataset)


Dataset({
    features: ['input'],
    num_rows: 100000
})


In [21]:
# Verificar os primeiros 3500 caracteres do arquivo formatado
with open(INPUT_PATH_DATASET, 'r', encoding='utf-8') as file:
    formatted_data = file.read(3500)
    print(formatted_data)

[
    {
        "input": "DESCRIBE THIS PRODUCTS.\n[|Title|] Girls Ballet Tutu Neon Pink[|eTitle|]\n\n[|Content|]High quality 3 layer ballet tutu. 12 inches in length[|eContent|]"
    },
    {
        "input": "DESCRIBE THIS PRODUCTS.\n[|Title|] Mog's Kittens[|eTitle|]\n\n[|Content|]Judith Kerr&#8217;s best&#8211;selling adventures of that endearing (and exasperating) cat Mog have entertained children for more than 30 years. Now, even infants and toddlers can enjoy meeting this loveable feline. These sturdy little board books&#8212;with their bright, simple pictures, easy text, and hand&#8211;friendly formats&#8212;are just the thing to delight the very young. Ages 6 months&#8211;2 years.[|eContent|]"
    },
    {
        "input": "DESCRIBE THIS PRODUCTS.\n[|Title|] Girls Ballet Tutu Neon Blue[|eTitle|]\n\n[|Content|]Dance tutu for girls ages 2-8 years. Perfect for dance practice, recitals and performances, costumes or just for fun![|eContent|]"
    },
    {
        "input": "DESCRIBE 

In [32]:
import json

# Gerar arquivo no formato alpaca:
#    formatted_data = {
#        "instruction": instructions,
#        "input": inputs,
#        "output": outputs
#    }

def format_dataset_into_model_input(data):
    def separate_text(full_text):
        # Separar as partes do texto com base nos delimitadores fornecidos
        title_start = full_text.find("[|Title|]") + len("[|Title|]")
        title_end = full_text.find("[|eTitle|]")
        content_start = full_text.find("[|Content|]") + len("[|Content|]")
        content_end = full_text.find("[|eContent|]")

        # Instrução é a primeira linha
        instruction = full_text.split('\n')[0].strip()
        input_text = full_text[title_start:title_end].strip()
        response = full_text[content_start:content_end].strip()

        return instruction, input_text, response

    # Inicializando as listas para armazenar os dados
    instructions = []
    inputs = []
    outputs = []

    # Processando o dataset
    for item in data:
        # Extrair o texto da chave "input"
        full_text = item["input"]
        instruction, input_text, response = separate_text(full_text)

        instructions.append(instruction)
        inputs.append(input_text)
        outputs.append(response)

    # Criando o dicionário final
    formatted_data = {
        "instruction": instructions,
        "input": inputs,
        "output": outputs
    }

    # Salvando o resultado em um arquivo JSON
    with open(OUTPUT_PATH_DATASET, 'w', encoding='utf-8') as output_file:
        json.dump(formatted_data, output_file, indent=4, ensure_ascii=False)

    print(f"Dataset salvo em {OUTPUT_PATH_DATASET}")

INPUT_PATH_DATASET = '/content/drive/MyDrive/finetuning/split_titles_dataset_1.json'
OUTPUT_PATH_DATASET = '/content/drive/MyDrive/finetuning/formatted_alpaca_dataset.json'

# Carregar o dataset original
with open(INPUT_PATH_DATASET, 'r', encoding='utf-8') as input_file:
    data = json.load(input_file)

# Chamar a função com o dataset carregado
format_dataset_into_model_input(data)


Dataset salvo em /content/drive/MyDrive/finetuning/formatted_alpaca_dataset.json


In [33]:
# Verificar os primeiros 500 caracteres do arquivo formatado
with open(OUTPUT_PATH_DATASET, 'r', encoding='utf-8') as file:
    formatted_data = file.read(500)
    print(formatted_data)

{
    "instruction": [
        "DESCRIBE THIS PRODUCTS.",
        "DESCRIBE THIS PRODUCTS.",
        "DESCRIBE THIS PRODUCTS.",
        "DESCRIBE THIS PRODUCTS.",
        "DESCRIBE THIS PRODUCTS.",
        "DESCRIBE THIS PRODUCTS.",
        "DESCRIBE THIS PRODUCTS.",
        "DESCRIBE THIS PRODUCTS.",
        "DESCRIBE THIS PRODUCTS.",
        "DESCRIBE THIS PRODUCTS.",
        "DESCRIBE THIS PRODUCTS.",
        "DESCRIBE THIS PRODUCTS.",
        "DESCRIBE THIS PRODUCTS.",
        "DESCRIBE THIS


In [34]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# Escolha do modelo pré-treinado que vamos utilizar
MODEL_NAME = "EleutherAI/gpt-neo-125M"

# Carregar o tokenizer e o modelo
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME)

In [35]:
from datasets import load_dataset

# Caminho para o arquivo JSON formatado
ALPACA_DATASET_PATH = '/content/drive/MyDrive/finetuning/formatted_alpaca_dataset.json'

# Carregar o dataset no formato JSON
dataset = load_dataset('json', data_files=ALPACA_DATASET_PATH)


Generating train split: 0 examples [00:00, ? examples/s]

In [28]:
# Verificar se o número de entradas é consistente entre colunas
input_lengths = len(dataset['train']['input'])
instruction_lengths = len(dataset['train']['instruction'])
output_lengths = len(dataset['train']['output'])

print(f"Length of 'input': {input_lengths}")
print(f"Length of 'instruction': {instruction_lengths}")
print(f"Length of 'output': {output_lengths}")

# Exibir algumas amostras do dataset carregado
print(dataset)
print(dataset['train'][0])  # Exemplo da primeira amostra


Length of 'input': 100000
Length of 'instruction': 100000
Length of 'output': 100000
DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 100000
    })
})
{'instruction': 'DESCRIBE THIS PRODUCTS.', 'input': 'Girls Ballet Tutu Neon Pink', 'output': 'High quality 3 layer ballet tutu. 12 inches in length'}


In [37]:
from datasets import load_dataset

# Caminho para o arquivo JSON formatado
ALPACA_DATASET_PATH = '/content/drive/MyDrive/finetuning/formatted_alpaca_dataset.json'

# Carregar o dataset no formato JSON
dataset = load_dataset('json', data_files=ALPACA_DATASET_PATH)

# Definir o token de padding como o token de fim de sequência (eos_token)
tokenizer.pad_token = tokenizer.eos_token

# Ou adicionar um novo token de padding
# tokenizer.add_special_tokens({'pad_token': '[PAD]'})

def tokenize_function(line):
    # Concatenar a instrução (instruction) e input
    prompt = line['instruction']

    # Garantir que o 'input' seja concatenado corretamente
    if isinstance(line['input'], list):
        # Se for uma lista, unir os elementos com espaço
        input_text = " ".join(line['input'])
    else:
        # Se for uma string, usar diretamente
        input_text = line['input']

    # Concatenar o input ao prompt (se o input não for vazio)
    if input_text:
        prompt += " " + input_text

    # Adicionar a resposta como rótulo (output)
    labels = line['output']

    # Tokenizar o prompt (entrada) e a resposta (saída)
    tokenized_line = tokenizer(prompt, truncation=True, padding='max_length', max_length=512)

    # Tokenizar a resposta e usá-la como labels
    tokenized_line['labels'] = tokenizer(labels, truncation=True, padding='max_length', max_length=512)['input_ids']

    return tokenized_line

# Aplicar a função de tokenização ao dataset
tokenized_datasets = dataset.map(tokenize_function, batched=True)



Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

ArrowInvalid: Column 1 named input expected length 44809 but got length 1000